# Keras Model Implementation

In [40]:
# Import Library
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.metrics import f1_score
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from src.cnn.keras_model import KerasModel
from src.cnn.cnn import CNN

In [41]:
# Config
EPOCHS = 10
BATCH_SIZE = 64
RANDOM_SEED = 42
RESULTS_BASE_DIR = "output/keras"

# Set seeds for reproducibility
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [42]:
if not os.path.exists(RESULTS_BASE_DIR):
    os.makedirs(RESULTS_BASE_DIR, exist_ok=True)
    print(f"Created directory for training results: {RESULTS_BASE_DIR}")

print("\nLoading CIFAR-10 dataset for training and testing")
(x_train_keras, y_train_keras), (x_test_keras, y_test_keras) = keras.datasets.cifar10.load_data()

input_shape_global = x_train_keras.shape[1:]

# Used for softmax in Dense Layer for the Keras
num_classes_global = 10

print(f"Data loaded: train shape: {x_train_keras.shape}, input_shape for model: {input_shape_global}")

standard_dense_config_train = [{'units': 128, 'dropout': 0.5}]
standard_global_pooling_train = 'global_avg'


Loading CIFAR-10 dataset for training and testing
Data loaded: train shape: (50000, 32, 32, 3), input_shape for model: (32, 32, 3)


In [43]:
# It defines the function to run training experiments
def run_experiment_set(experiment_group_name, model_configs_list, input_shape, num_classes, x_train_full_data, y_train_full_data, x_test_data, y_test_data):
    print(f"\n{'='*20} Starting TRAINING Experiment Group: {experiment_group_name} {'='*20}")
    group_results = []
    
    shared_model_handler_for_data = KerasModel(input_shape=input_shape, num_class=num_classes, random_seed=RANDOM_SEED)
    
    shared_model_handler_for_data.preprocess_data(
        x_train_full_data, y_train_full_data, x_test_data, y_test_data
    )

    for config_details in model_configs_list:
        model_name = config_details['name']
        model_params = config_details['params']

        print(f"\n--- Training Configuration: {model_name} ---")
        
        current_model_handler = KerasModel(input_shape=input_shape, num_class=num_classes, random_seed=RANDOM_SEED)
        
        current_model_handler.x_train = shared_model_handler_for_data.x_train
        current_model_handler.y_train = shared_model_handler_for_data.y_train
        current_model_handler.x_val = shared_model_handler_for_data.x_val
        current_model_handler.y_val = shared_model_handler_for_data.y_val
        current_model_handler.x_test_processed = shared_model_handler_for_data.x_test_processed
        current_model_handler.y_test_processed = shared_model_handler_for_data.y_test_processed

        current_model_handler.define_model(
            model_name=model_name,
            conv_blocks_config=model_params['conv_blocks_config'],
            global_pooling_type=model_params['global_pooling_type'],
            dense_layers_config=model_params['dense_layers_config']
        )
        current_model_handler.compile_model()
        
        current_model_handler.train_model(epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
        
        loss, acc, f1 = current_model_handler.evaluate_model(verbose=0)
        
        weights_path = os.path.join(RESULTS_BASE_DIR, experiment_group_name, model_name, f"{model_name}_weights.weights.h5")
        current_model_handler.save_model_weights(weights_path) 
        
        current_model_handler.plot_training_history(experiment_group_name, model_name, base_save_path=RESULTS_BASE_DIR)
        
        group_results.append({
            'model_name': model_name, 'f1_score': f1, 'accuracy': acc,
            'loss': loss, 'config_params': model_params
        })

    sorted_results = sorted(group_results, key=lambda x: x['f1_score'], reverse=True)
    for res in sorted_results:
        print(f"Model: {res['model_name']}, F1-Score: {res['f1_score']:.4f}, Accuracy: {res['accuracy']:.4f}")

    return sorted_results

In [44]:
training_configurations = []
experiments_to_run_grouped = {}
all_training_results = {}


if 'all_individual_training_runs_summary' not in locals(): 
    all_individual_training_runs_summary = {}

def read_training_config(group_configs):
    """
    Processes a list of model configurations for a specific experiment group.
    For each model configuration in the list:
    - Checks if weights exist.
    - If not, calls run_experiment_set to train that single model.
    """
    if not isinstance(group_configs, list):
        print(f"ERROR in read_training_config: Expected a list of configurations, but got {type(group_configs)}")
        return

    for single_model_config_dict in group_configs:
        model_name = single_model_config_dict['name']

        experiment_group = single_model_config_dict['experiment_group']
        model_params = single_model_config_dict['params'] # The actual architecture

        print(f"\nChecking/Processing config: '{model_name}' in group '{experiment_group}'")
        
        expected_weights_path = os.path.join(RESULTS_BASE_DIR, experiment_group, model_name, f"{model_name}_weights.weights.h5")
        
        if os.path.exists(expected_weights_path):
            print(f"    Weights for '{model_name}' already exist at '{expected_weights_path}'. Skipping training.")
            all_individual_training_runs_summary[model_name] = {
                'status': 'skipped_weights_exist', 
                'weights_path': expected_weights_path,
                'experiment_group': experiment_group 
            }
        else:
            print(f"    Weights for '{model_name}' NOT found. Proceeding with training for this model...")
        
            config_for_runner = [{'name': model_name, 'params': model_params}]
    
            individual_run_results_list = run_experiment_set(
                experiment_group_name=experiment_group, 
                model_configs_list=config_for_runner, 
                input_shape=input_shape_global,
                num_classes=num_classes_global,
                x_train_full_data=x_train_keras,
                y_train_full_data=y_train_keras,
                x_test_data=x_test_keras,        
                y_test_data=y_test_keras         
            )
            # run_experiment_set returns a list of results, even for one model
            if individual_run_results_list:
                all_individual_training_runs_summary[model_name] = individual_run_results_list[0]
            
                if 'experiment_group' not in all_individual_training_runs_summary[model_name]:
                     all_individual_training_runs_summary[model_name]['experiment_group'] = experiment_group
            else:
                all_individual_training_runs_summary[model_name] = {
                    'experiment_group': experiment_group
                }
    print(f"Finished processing group of configurations")

In [45]:
# Training A (Comparing in Jumlah layer Konvolusi)
group_A_name_train = "A_Jumlah_Layer_Konvolusi"

Conv1 = [{'filters': 32, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25}]

Conv2 = [{'filters': 32, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25},
            {'filters': 64, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25}]
Conv3 = [ {'filters': 32, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25},
            {'filters': 64, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25},
            {'filters': 128, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25}]

training_A_Config = [
    {'name': 'A1_1_ConvBlock', 'experiment_group': group_A_name_train, 'params': {
        'conv_blocks_config': Conv1,
        'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
    {'name': 'A2_2_ConvBlocks', 'experiment_group': group_A_name_train, 'params': {
        'conv_blocks_config':Conv2,
        'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
    {'name': 'A3_3_ConvBlocks', 'experiment_group': group_A_name_train, 'params': {
        'conv_blocks_config': Conv3,
        'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
]

training_configurations.extend(training_A_Config)
read_training_config(training_A_Config)


Checking/Processing config: 'A1_1_ConvBlock' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A1_1_ConvBlock' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A1_1_ConvBlock\A1_1_ConvBlock_weights.weights.h5'. Skipping training.

Checking/Processing config: 'A2_2_ConvBlocks' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A2_2_ConvBlocks' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A2_2_ConvBlocks\A2_2_ConvBlocks_weights.weights.h5'. Skipping training.

Checking/Processing config: 'A3_3_ConvBlocks' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A3_3_ConvBlocks' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A3_3_ConvBlocks\A3_3_ConvBlocks_weights.weights.h5'. Skipping training.
Finished processing group of configurations


In [46]:
# Training B (Comparing the amount of Filter)
group_B_name_train = "B_Banyak_Filter"

def create_conv_forB(filters_list):
    return [{'filters': filters_list[0], 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25},
            {'filters': filters_list[1], 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25}]

training_B_Config = [
    {'name': 'B1_Filters_16_32', 'experiment_group': group_B_name_train, 'params': 
     {'conv_blocks_config': create_conv_forB([16, 32]), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
    {'name': 'B2_Filters_32_64', 'experiment_group': group_B_name_train, 'params': 
     {'conv_blocks_config': create_conv_forB([32, 64]), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
    {'name': 'B3_Filters_64_128', 'experiment_group': group_B_name_train, 'params': 
     {'conv_blocks_config': create_conv_forB([64, 128]), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
]

training_configurations.extend(training_B_Config)
read_training_config(training_configurations)


Checking/Processing config: 'A1_1_ConvBlock' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A1_1_ConvBlock' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A1_1_ConvBlock\A1_1_ConvBlock_weights.weights.h5'. Skipping training.

Checking/Processing config: 'A2_2_ConvBlocks' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A2_2_ConvBlocks' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A2_2_ConvBlocks\A2_2_ConvBlocks_weights.weights.h5'. Skipping training.

Checking/Processing config: 'A3_3_ConvBlocks' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A3_3_ConvBlocks' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A3_3_ConvBlocks\A3_3_ConvBlocks_weights.weights.h5'. Skipping training.

Checking/Processing config: 'B1_Filters_16_32' in group 'B_Banyak_Filter'
    Weights for 'B1_Filters_16_32' already exist at 'output/keras\B_Banyak_Filter\B1_Filters_16_32\B1_Filters_16_32_weights.weights.h5'. Skipping training.

Checking/Processing config: 'B2_Filte

In [47]:
# Training C (Comparing the Size of Kernel)
group_C_name_train = "C_Ukuran_Filter"
def create_conv_forC(kernel_sizes_list):
    return [{'filters': 64, 'kernel_size': kernel_sizes_list[0], 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25},
            {'filters': 64, 'kernel_size': kernel_sizes_list[1], 'conv_layers_in_block': 1, 'pooling_type': 'max', 'dropout_after_pool': 0.25}]

training_C_config = [
    {'name': 'C1_Kernel_2x2', 'experiment_group': group_C_name_train, 'params': 
     {'conv_blocks_config': create_conv_forC([(2,2), (2,2)]), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
    {'name': 'C2_Kernel_3x3', 'experiment_group': group_C_name_train, 'params': 
     {'conv_blocks_config': create_conv_forC([(3,3), (3,3)]), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
    {'name': 'C3_Kernel_5x5', 'experiment_group': group_C_name_train, 'params': 
     {'conv_blocks_config': create_conv_forC([(5,5), (5,5)]), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
]

training_configurations.extend(training_C_config)
read_training_config(training_configurations)


Checking/Processing config: 'A1_1_ConvBlock' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A1_1_ConvBlock' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A1_1_ConvBlock\A1_1_ConvBlock_weights.weights.h5'. Skipping training.

Checking/Processing config: 'A2_2_ConvBlocks' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A2_2_ConvBlocks' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A2_2_ConvBlocks\A2_2_ConvBlocks_weights.weights.h5'. Skipping training.

Checking/Processing config: 'A3_3_ConvBlocks' in group 'A_Jumlah_Layer_Konvolusi'
    Weights for 'A3_3_ConvBlocks' already exist at 'output/keras\A_Jumlah_Layer_Konvolusi\A3_3_ConvBlocks\A3_3_ConvBlocks_weights.weights.h5'. Skipping training.

Checking/Processing config: 'B1_Filters_16_32' in group 'B_Banyak_Filter'
    Weights for 'B1_Filters_16_32' already exist at 'output/keras\B_Banyak_Filter\B1_Filters_16_32\B1_Filters_16_32_weights.weights.h5'. Skipping training.

Checking/Processing config: 'B2_Filte

In [48]:
# Training D (Comparing the type of the pooling)
group_D_name_train = "D_Jenis_Pooling"
def create_conv_forD(pooling_type_str):
    return [{'filters': 32, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': pooling_type_str, 'pooling_size': (2,2), 'dropout_after_pool': 0.25},
            {'filters': 64, 'kernel_size': (3,3), 'conv_layers_in_block': 1, 'pooling_type': pooling_type_str, 'pooling_size': (2,2), 'dropout_after_pool': 0.25}]
training_D_config = [
    {'name': 'D1_Pooling_Max', 'experiment_group': group_D_name_train, 'params': 
     {'conv_blocks_config': create_conv_forD('max'), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
    {'name': 'D2_Pooling_Avg', 'experiment_group': group_D_name_train, 'params': 
     {'conv_blocks_config': create_conv_forD('avg'), 'global_pooling_type': standard_global_pooling_train, 'dense_layers_config': standard_dense_config_train}},
]

training_configurations.extend(training_D_config)
read_training_config(training_D_config)


Checking/Processing config: 'D1_Pooling_Max' in group 'D_Jenis_Pooling'
    Weights for 'D1_Pooling_Max' already exist at 'output/keras\D_Jenis_Pooling\D1_Pooling_Max\D1_Pooling_Max_weights.weights.h5'. Skipping training.

Checking/Processing config: 'D2_Pooling_Avg' in group 'D_Jenis_Pooling'
    Weights for 'D2_Pooling_Avg' already exist at 'output/keras\D_Jenis_Pooling\D2_Pooling_Avg\D2_Pooling_Avg_weights.weights.h5'. Skipping training.
Finished processing group of configurations


In [49]:
def create_keras_model_for_loading(model_name, input_s, num_c, architecture_config):
    """
    Reconstructs a Keras model based on an architecture configuration.
    This is used for loading weights 
    """
    conv_blocks_config = architecture_config['conv_blocks_config']
    global_pooling_type = architecture_config['global_pooling_type']
    dense_layers_config = architecture_config['dense_layers_config']

    model_layers_list = [layers.Input(shape=input_s, name="input_layer")]
    
    for i, block_config in enumerate(conv_blocks_config):
        for j in range(block_config['conv_layers_in_block']):
            model_layers_list.append(layers.Conv2D(
                filters=block_config['filters'], kernel_size=block_config['kernel_size'],
                activation='relu', 
                padding='same', name=f"conv_block{i+1}_layer{j+1}"
            ))
        if block_config['pooling_type'] == 'max':
            model_layers_list.append(layers.MaxPooling2D(pool_size=block_config.get('pooling_size', (2,2)), name=f"maxpool_block{i+1}"))
        elif block_config['pooling_type'] == 'avg':
             model_layers_list.append(layers.AveragePooling2D(pool_size=block_config.get('pooling_size', (2,2)), name=f"avgpool_block{i+1}"))
    
    if global_pooling_type == 'global_avg': 
        model_layers_list.append(layers.GlobalAveragePooling2D(name="global_avg_pool_layer"))

    elif global_pooling_type == 'flatten': 
        model_layers_list.append(layers.Flatten(name="flatten_layer"))
    
    for i, dense_config_item in enumerate(dense_layers_config):
        model_layers_list.append(layers.Dense(dense_config_item['units'], activation='relu', name=f"dense_layer{i+1}"))
            
    model_layers_list.append(layers.Dense(num_c, activation='softmax', name="output_layer"))
    
    keras_model = keras.Sequential(model_layers_list, name=model_name + "_keras_loaded_for_inference")
    return keras_model

In [50]:
def run_forward_propagation_comparison(
    model_name_tag,
    keras_architecture_config, 
    base_weights_dir_to_load, 
    experiment_group_for_weights,
    x_test_full_keras_data,    
    y_test_full_keras_data,    
    input_shape_param,        
    num_classes_param,        
    num_samples_to_test=50 
):
    '''
    This function used to compare the prediction between the keras and scratch
    '''
    print(f"\nProcessing Model: {model_name_tag} (from group: {experiment_group_for_weights})")

    results = {
        'model_name': model_name_tag, 'f1_keras': None, 'f1_scratch': None,
        'f1_difference': None, 'label_matches_ratio': None,
        'mean_abs_prob_diff': None, 'status': 'Not Processed'
    }

    keras_weights_path = os.path.join(base_weights_dir_to_load, experiment_group_for_weights, model_name_tag, f"{model_name_tag}_weights.weights.h5")
    
    if not os.path.exists(keras_weights_path):
        return results

    print(f"Reconstructing Keras model '{model_name_tag}' for inference...")
    keras_model_loaded = create_keras_model_for_loading(
        model_name_tag, input_shape_param, num_classes_param, keras_architecture_config
    )
    try:
        keras_model_loaded.load_weights(keras_weights_path)
    except Exception as e:
        print(f"ERROR: Failed to load Keras for {model_name_tag}. Error: {e}")
        return results

    x_test_normalized_all = x_test_full_keras_data.astype("float32") / 255.0
    y_test_labels_all_flat = y_test_full_keras_data.flatten()
    
    actual_num_samples = min(num_samples_to_test, len(x_test_normalized_all))
    x_test_subset = x_test_normalized_all[:actual_num_samples]
    y_test_subset_labels = y_test_labels_all_flat[:actual_num_samples]
    if actual_num_samples == 0:
        print("ERROR : No S")
        return results
    print(f"Using {actual_num_samples} samples for prediction testing.")

    scratch_model_instance = CNN()
    scratch_model_instance.load_keras_model(keras_model_loaded)

    print(f"Keras Prediction : ")
    print()

    keras_probabilities = keras_model_loaded.predict(x_test_subset, verbose=0)
    keras_predictions = np.argmax(keras_probabilities, axis=1)
    results['f1_keras'] = f1_score(y_test_subset_labels, keras_predictions, average='macro', zero_division=0)
    print(f"Keras F1-Score: {results['f1_keras']:.4f}")

    print(f"CNN Scratch Prediction : ")
    if scratch_model_instance.layers:
        scratch_probabilities = scratch_model_instance.predict_batch(x_test_subset, model_name_tag)
        scratch_predictions = np.argmax(scratch_probabilities, axis=1)
        results['f1_scratch'] = f1_score(y_test_subset_labels, scratch_predictions, average='macro', zero_division=0)
        print(f"Scratch F1-Score: {results['f1_scratch']:.4f}")
    else:
        print("Scratch model has no layers, skipping Scratch prediction.")
        return results 

    
    if results['f1_keras'] is not None and results['f1_scratch'] is not None:
        results['f1_difference'] = abs(results['f1_keras'] - results['f1_scratch'])
        num_matching_labels = np.sum(keras_predictions == scratch_predictions)
        results['label_matches_ratio'] = num_matching_labels / actual_num_samples
        results['mean_abs_prob_diff'] = np.mean(np.abs(keras_probabilities - scratch_probabilities))

        print(f"F1-Score Difference: {results['f1_difference']:.6f}")
        print(f"Label Match Ratio: {results['label_matches_ratio']:.4f}")
        print(f"Mean Absolute Probability Difference: {results['mean_abs_prob_diff']:.6f}")
        
        if results['f1_difference'] < 1e-3 and results['label_matches_ratio'] >= 0.99: 
            print(f"Status: Success - Results Closely Match")
        else:
            print(f"Status: WARNING - Significant Difference in Results") 

    print(f"Finished processing Model: {model_name_tag}")
    return results

In [51]:
# Run the Comparison

NUM_SAMPLES_FOR_FWD_PROP_TEST = 50

if 'training_configurations' not in locals() or not training_configurations:
    print("ERROR: training configurations is empty")
    forward_prop_test_configs_adapted = []
else:
    forward_prop_test_configs_adapted = []
    for train_config_item in training_configurations:
        forward_prop_test_configs_adapted.append({
            'model_name_tag': train_config_item['name'],
            'experiment_group_for_weights': train_config_item['experiment_group'], 
            'architecture_config': train_config_item['params'] 
        })
    print(f"Total configurations adapted for forward-prop test: {len(forward_prop_test_configs_adapted)}")



all_comparison_results = []
num_models_to_process_fp = len(forward_prop_test_configs_adapted) 

if num_models_to_process_fp > 0:
    print(f"\nStarting forward propagation comparison for {num_models_to_process_fp} model configurations")

    for i, model_config_entry in enumerate(forward_prop_test_configs_adapted):
        print(f"\nTest Forward Prop for Model {i+1}/{num_models_to_process_fp}")
        
        result_dict = run_forward_propagation_comparison(
            model_name_tag= model_config_entry['model_name_tag'],
            keras_architecture_config= model_config_entry['architecture_config'],
            base_weights_dir_to_load= RESULTS_BASE_DIR,
            experiment_group_for_weights= model_config_entry['experiment_group_for_weights'],
            x_test_full_keras_data= x_test_keras,
            y_test_full_keras_data= y_test_keras,
            input_shape_param= input_shape_global, 
            num_classes_param= num_classes_global,    
            num_samples_to_test= NUM_SAMPLES_FOR_FWD_PROP_TEST
        )
        all_comparison_results.append(result_dict)
    print("\n\nAll forward propagation comparisons finished.")
else:
    print("No model configurations available to test for forward propagation.")

Total configurations adapted for forward-prop test: 11

Starting forward propagation comparison for 11 model configurations

Test Forward Prop for Model 1/11

Processing Model: A1_1_ConvBlock (from group: A_Jumlah_Layer_Konvolusi)
Reconstructing Keras model 'A1_1_ConvBlock' for inference...
Using 50 samples for prediction testing.
Load Keras Model 
Adding Conv2DLayer (kernel: (3, 3, 3, 32), stride: (1, 1), padding: same)
Adding ReLULayer
Adding MaxPooling2DLayer (pool_size: (2, 2), stride: (2, 2))
Adding GlobalAveragePooling2DLayer
Adding Dense Layer(weight: (32, 128))
Adding ReLu
Adding Dense Layer(weight: (128, 10))
Adding SoftMax
Keras Prediction : 

Keras F1-Score: 0.4405
CNN Scratch Prediction : 

Starting Scratch Prediction for 'A1_1_ConvBlock' on 50 samples:
Processing: 50/50 (100.0%)
Batch prediction complete for 'A1_1_ConvBlock'.
Total time: 7.47 seconds (0.149 sec/sample).
Output array shape: (50, 10)

Scratch F1-Score: 0.4405
F1-Score Difference: 0.000000
Label Match Ratio: 

In [52]:
if all_comparison_results:
    df_results = pd.DataFrame(all_comparison_results)
    column_order = ['model_name',  'f1_keras', 'f1_scratch']
    for col in column_order:
        if col not in df_results.columns:
            df_results[col] = None      
    df_results = df_results[column_order] 
    
    print("\nSummary of Forward Propagation Comparison Results")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 120) 
    pd.set_option('display.float_format', '{:.6f}'.format)
    
    print(df_results)
else:
    print("No comparison results to display.")


Summary of Forward Propagation Comparison Results
           model_name  f1_keras  f1_scratch
0      A1_1_ConvBlock  0.440476    0.440476
1     A2_2_ConvBlocks  0.476779    0.476779
2     A3_3_ConvBlocks  0.691220    0.691220
3    B1_Filters_16_32  0.499015    0.499015
4    B2_Filters_32_64  0.467274    0.467274
5   B3_Filters_64_128  0.599246    0.599246
6       C1_Kernel_2x2  0.445812    0.416606
7       C2_Kernel_3x3  0.473990    0.473990
8       C3_Kernel_5x5  0.615709    0.615709
9      D1_Pooling_Max  0.497829    0.497829
10     D2_Pooling_Avg  0.430238    0.341209
